In [ ]:
from pylsl import StreamInlet, resolve_stream
import numpy as np
import joblib  # Used for loading sklearn models
import sys
import os
import torch

sys.path.append('./src/processing')
from preprocessing import *

# sys.path.append('./models')
# from eegconformer import EEGConformer

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


: 

In [ ]:
models_dir = './models/trained/'
results_dir = './results/'

# Configuration
srate = 160  #Sampling rate of the EEG data
epoch_length_sec = 5  # Length of the desired sample in seconds
samples_needed = srate * epoch_length_sec  # Number of samples needed for ~5 seconds
# Manually define from eegbci dataset
channel_names = ['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FT8', 'T7', 'T8', 'T9', 'T10', 'TP7', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']


In [3]:

print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG')

def collect_and_save_single_sample(inlet, samples_needed):
    '''
    Test function to collect and save a single sample of EEG data
    :param inlet: The LSL StreamInlet object
    :param samples_needed: The number of samples to collect
    :return: None
    '''
    buffer = []  # Initialize the buffer to hold collected samples
    timestamps = []  # To store timestamps of each sample

    while len(buffer) < samples_needed:
        # Continuously pull samples
        sample, timestamp = inlet.pull_sample()
        if sample:
            buffer.append(sample)  # Add the sample to the buffer
            timestamps.append(timestamp)  # Add the timestamp

        if len(buffer) >= samples_needed:
            # Once we have enough samples, save and exit
            np.save('sample.npy', np.array(buffer))  # Save the buffer as a numpy file
            print(f"Saved ~{epoch_length_sec}-second sample with {len(buffer)} samples.")
            return  # Exit the function, effectively stopping data collection

# Call the function to collect, save, and then stop
collect_and_save_single_sample(inlet, samples_needed)


Looking for an EEG stream...


2024-03-12 22:15:03.968 (   5.808s) [          488C77]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-12 22:15:03.969 (   5.809s) [          488C77]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-12 22:15:03.969 (   5.809s) [          488C77]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-03-12 22:15:03.969 (   5.809s) [          488C77]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-12 22:15:03.969 (   5.809s) [          488C77]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-03-12 22:15:03.969 (   5.809s) [          488C77]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-12 22:15:03.969 (   5.809s) [          488C77]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-03-12 22:15:03.969 (   5.809s) [          488C77]      netinterfaces.cpp:91    I

In [4]:
# # Test with one saved sample sent 
sample = np.load('sample.npy')
sample.shape

(800, 64)

In [5]:
preprocessed_data = preprocess_single_trial(sample, srate, channel_names)

Creating RawArray with float64 data, n_channels=64, n_times=800
    Range : 0 ... 799 =      0.000 ...     4.994 secs
Ready.


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 265 samples (1.656 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [8]:
# csp + lda decode
model_path = os.path.join(models_dir, 'csp_lda.pkl')
loaded_model = joblib.load(model_path)
predicted_labels = loaded_model.predict(preprocessed_data)
predicted_labels

# Save result

array([1])

In [9]:
# csp + logistic regression decode
model_path = os.path.join(models_dir, 'csp_logistic.pkl')
loaded_model = joblib.load(model_path)
predicted_labels = loaded_model.predict(preprocessed_data)
predicted_labels

array([0])

In [20]:
# csp + svm decode
model_path = os.path.join(models_dir, 'csp_svm.pkl')
loaded_model = joblib.load(model_path)
predicted_labels = loaded_model.predict(preprocessed_data)
predicted_labels

array([0])

In [21]:
_, n_chans, n_times = preprocessed_data.shape

In [22]:
# eeg_conformer decode
model = EEGConformer(
    n_outputs= 2,
    n_chans = n_chans,
    sfreq= srate,
    n_times = n_times,
    n_filters_time=40, 
    filter_time_length=25,
    pool_time_length=75,
    pool_time_stride=15,
    drop_prob=0.7,
    att_depth=3,
    att_heads=10,
    att_drop_prob=0.7,
    final_fc_length='auto', # could be 'auto' or int
    return_features=False, # returns the features before the last classification layer if True
    chs_info=None,
    input_window_seconds=None,
    add_log_softmax=True,
)

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
/Users/nana/Documents/GitHub/eeg/./models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [23]:
loaded_model = os.path.join(models_dir, 'cross_subject_conformer.pth')
checkpoint = torch.load(loaded_model)

In [24]:
model.load_state_dict(checkpoint)

In [1]:
samples_per_epoch = srate * epoch_length_sec

# Resolve the stream
print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG', 'name', 'BioSemi')
inlet = StreamInlet(streams[0])

# Initialize a buffer for accumulating samples
buffer = []

model_path = os.path.join(models_dir, 'csp_logistic.pkl')
loaded_model = joblib.load(model_path)

def decode(epoch, loaded_model, sample, srate, channel_names):
    """
    Process and decode an epoch of EEG data.
    """
    preprocessed_epoch = preprocess_single_trial(sample, srate, channel_names)
    predicted_labels = loaded_model.predict(preprocessed_data)
    return prediction

pred_hist = []
while True:
    # Pull sample from LSL stream
    sample, timestamp = inlet.pull_sample()
    buffer.append(sample)
    
    # Check if buffer has enough samples to form an epoch
    if len(buffer) >= samples_per_epoch:
        epoch = np.array(buffer[:samples_per_epoch])  
        buffer = buffer[samples_per_epoch:]  
        
        # Decode the epoch
        prediction = decode(epoch)
        pred_hist.append((timestamp, prediction))
        print(f"Timestamp: {timestamp}, Prediction: {prediction}")


NameError: name 'srate' is not defined